In [3]:
# 載入套件
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re

from wordcloud import WordCloud # pip install wordcloud
import matplotlib.pyplot as plt
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda
# 設定圖的字體大小
plt.rcParams['font.size'] = 14

# 設定中文字體 (無法顯示的話可以試試‘Microsoft JhengHei’字體)
# 也可參考：https://pyecontech.com/2020/03/27/python_matplotlib_chinese/
plt.rcParams['font.sans-serif'] = ['Arial Unicode Ms']

c:\Users\win90\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
maildata = pd.read_csv("dataset/Phishing_Email.csv")

# 刪除標點符號/數字/換行符號
maildata["Email Text"] = maildata["Email Text"].apply(lambda x: re.sub(r'[^\w\s]','', str(x)))  #只留下英文字母和空格(包含換行符號)
maildata["Email Text"] = maildata["Email Text"].apply(lambda x: re.sub(r'[\n_-]+',' ', x)) #將換行符號替換成空格
maildata["Email Text"] = maildata["Email Text"].apply(lambda x: re.sub(r'\s+', ' ', x)) # 將多個空格替換成一個空格
# 新增一個 column，計算 "Email Text" 的字數
maildata["Word Count"] = maildata["Email Text"].apply(lambda x: len(str(x)))

# 字數介於 100 到 600 之間的資料
# 避免 token 太多
maildata = maildata[maildata["Word Count"] > 100]
maildata = maildata[maildata["Word Count"] < 600]

In [5]:
nums = 25
# 隨機抽取 Phishing Email
phishing_emails = maildata[maildata['Email Type'] == 'Phishing Email'].sample(n=nums, random_state=1116)
# 隨機抽取 Safe Email
safe_emails = maildata[maildata['Email Type'] == 'Safe Email'].sample(n=nums, random_state=1116)

### 利用 huggingFace + OpenAI 使用 LLM

In [6]:
import os
import getpass
import pandas as pd
from glob import glob
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.tools import Tool
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories import FileChatMessageHistory

from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [7]:
# 輸入 OpenAI 的 token
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [8]:
# 載入模型
# chatmodel 更適合當聊天機器人
llm = ChatOpenAI(
    model_name="gpt-4.1-nano",
    temperature=0.1,
    streaming=True,
    max_tokens=300 
)

C:\Users\win90\AppData\Local\Temp\ipykernel_23420\3540564313.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [9]:
# 計算 accuracy
def calcu_result(result_list):
    true_count = result_list.count("True")
    false_count = result_list.count("False")
    return true_count / (true_count + false_count)

### Normal

In [54]:
SystemPrompt = SystemMessagePromptTemplate.from_template(
"""
你是一位資安專家，將會幫助判斷以下郵件是否為釣魚信件。
如果是釣魚信件，請回答「True」，如果不是釣魚信件，請回答「False」。
"""
)

HumanPrompt = HumanMessagePromptTemplate.from_template(
"""
郵件：{input}
"""
)

normal_prompt = ChatPromptTemplate.from_messages([
    SystemPrompt,
    HumanPrompt,
    # MessagesPlaceholder(variable_name="agent_scratchpad")
])

normal_chain = (
    {"input": RunnablePassthrough()}
    | normal_prompt
    | llm
)

In [55]:
# 將結果存入 list
normal_result = []
normal_ans = []   
# 使用for迴圈遍歷phishing_emails的"Email Text"欄位
# 將是否答對存進 list 中，而非回答的答案

for email_text in phishing_emails["Email Text"]:
    result = normal_chain.invoke({"input": email_text})
    result = result.content
    normal_result.append(result)
    if "true" in result.strip().lower():
        normal_ans.append("True")
    else:
        normal_ans.append("False")

for email_text in safe_emails["Email Text"]:
    result = normal_chain.invoke({"input": email_text})
    result = result.content
    normal_result.append(result)
    if "false" in result.strip().lower():
        normal_ans.append("True")
    else:
        normal_ans.append("False")

In [56]:
normal_acc = calcu_result(normal_ans)
print(normal_acc)

0.72


In [57]:
print(normal_result)

['True', 'True', 'False', 'True', 'False', 'True', 'True', 'False', 'True', 'True', 'True', 'True', 'True', 'True', 'False', 'True', 'True', 'False', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'False', 'False', 'False', 'False', 'True', 'True', 'False', 'False', 'False', 'False', 'True', 'False', 'True', 'False', 'False', 'True', 'True', 'False', 'True', 'False', 'False', 'False', 'True', 'False', 'True']


### Few shot

In [23]:
SystemPrompt = SystemMessagePromptTemplate.from_template(
"""
你是一位資安專家，將會幫助判斷以下郵件是否為釣魚信件。
如果是釣魚信件，請回答「True」，如果不是釣魚信件，請回答「False」。

以下為一些參考例子：
```
郵件：hot teen flasher sex spy over 100  000 voyeur images tons live hidden cams 50  000  extreme movies total invasion of privacy click here for more  sex spy is the number one site for complete invasion of privacy photos  videos and more  these extreme voyeur photos have the hottest women caught on tape doing unthinkable acts  these sluts have no clue we are watching  click here for a free preview of the dumbest whores getting caught on camera  remove me ashtray bertrand ductwork witt secretion lucre lundberg incurring nostalgic papua yakima
答案：True

郵件：2002 it systems development allocations kevin tim please review the attached listing and advise if you agree to the projects and amounts listed as billings that enron north america will receive in 2002 for support of these efforts  i  m at x 30352 if you have any questions  thanks 
答案：False

郵件：enrononline executive summary for november 08 2001 following please find the daily enrononline executive summary  note  the executive summary transaction counts have been reduced to reflect the removal of sleeve trade activity 
答案：False
"""
)

HumanPrompt = HumanMessagePromptTemplate.from_template(
"""
郵件：{input}
"""
)

fewshot_prompt = ChatPromptTemplate.from_messages([
    SystemPrompt,
    HumanPrompt,
    # MessagesPlaceholder(variable_name="agent_scratchpad")
])

fewshot_chain = (
    {"input": RunnablePassthrough()}
    | fewshot_prompt
    | llm
)

In [58]:
# 將結果存入 list
fewshot_result = []
fewshot_ans = []   
# 使用for迴圈遍歷phishing_emails的"Email Text"欄位
# 將是否答對存進 list 中，而非回答的答案

for email_text in phishing_emails["Email Text"]:
    result = fewshot_chain.invoke({"input": email_text})
    result = result.content
    fewshot_result.append(result)
    if "true" in result.strip().lower():
        fewshot_ans.append("True")
    else:
        fewshot_ans.append("False")

for email_text in safe_emails["Email Text"]:
    result = fewshot_chain.invoke({"input": email_text})
    result = result.content
    fewshot_result.append(result)
    if "false" in result.strip().lower():
        fewshot_ans.append("True")
    else:
        fewshot_ans.append("False")

In [59]:
fewshot_acc = calcu_result(fewshot_ans)
print(fewshot_acc)

0.66


In [60]:
print(fewshot_result)

['True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'False', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'False', 'False', 'False', 'True', 'True', 'True', 'False', 'True', 'False', 'True', 'True', 'True', 'False', 'False', 'True', 'True', 'True', 'True', 'False', 'True', 'True', 'True', 'False', 'True']


### CoT

In [35]:
SystemPrompt = SystemMessagePromptTemplate.from_template(
"""
你是一位資安專家，將會幫助判斷以下郵件是否為釣魚信件。
請逐步分析郵件的內容，並按照以下流程進行：
1. 提取郵件的主旨與目的。
2. 分析郵件的語言特徵（例如是否使用過於吸引人的措辭、過多的呼籲性語句）。
3. 判斷是否包含潛在的釣魚信件特徵（例如要求用戶提供敏感信息、提供不可信的 URL 或強調緊急性）。
4. 根據分析給出結論，請先回答「True」或「False」，再提供解釋。


以下為一些例子：
```
郵件：HI As a professional bulk mailer for 5 years I made over 200000 last 12 months selling only one product The Banned CDLuckily for you the Bulker CD 2003 is now for sale I decide to sell all secrets how I made over 20000 per month at home only few hours workWhy dont more people use Bulk Email Marketing1 Lack of knowledge Most people do not know how to set up a marketing campaign let alone set up an effective email marketing campaign Through hard work and trial and error we have developed simple yet successful strategies to send your emails We can show you how to do it properly 2 Fear of getting into trouble Most people do not send email because they have heard negative things about SPAM and that your isp will shut you down This is true if you dont know what you are doing and bulk email to the masses If you dont believe in SPAM we have developed alternative ways to bulk email so that you are sending your emails responsibly without getting into any trouble at all 3 Dont have the necessary equipmentsoftwares To send your emails out you need a computer with specialized email software installed that will send or harvest your emails We are the email marketing software experts The softwares ranging will up to thousands of dollars Buying the correct software for your needs can be confusing Depending on your budget requirements and goals we can help recommend the best software for youBULKER CDROM has everything you need to start bulk Emailing immediately all on this one CD BULKER CDROM is excellent for the beginner as well as the professional Advertising my products have never been easier Please Click the URL for the Detail http7777772E62a6E6E6564c642E6E6574bulkhtmYou are receiving this special offer because you have provided permission to receive third party email communications regarding special online promotions or offers We strongly oppose the use of SPAM email and do not want to send our mailings to anyone who does not wish to receive them If you do not wish to receive any further messages from netcommission To Be Removed From Our List http7777772E62a6E6E6564c642E6E6574removehtml
分析：
1. 主旨與目的：該郵件的主要目的是推銷一款名為 "Bulker CDROM" 的軟件，並強調其能幫助用戶實現有效的電子郵件營銷。
2. 語言特徵：郵件使用了吸引人的措辭（例如「我一年內賺了200000美元」、「僅需幾個小時的工作」），並帶有誘惑性言語來吸引目標群體。它還通過羅列「為什麼人們不用批量電子郵件行銷」的原因，試圖建立信任。
3. 潛在釣魚信特徵：
- 提供了 URL：http7777772E62a6E6E6564c642E6E6574bulkhtm，這看起來並非可信且可能存在安全風險。
- 試圖說服用戶購買產品以獲得「快速致富」的方案。
- 提到如果不希望收到郵件，可點擊移除鏈接，但鏈接的真實性無法確認，可能會收集用戶敏感信息。
4. 結論：「True」，該郵件包含了典型的釣魚信件特徵，包括不可信的 URL 和過於誘惑的語言，因此判斷為釣魚信件（True）。

郵件：2002 it systems development allocations kevin  tim  please review the attached listing and advise if you agree to the projects and amounts listed as billings that enron north america will receive in 2002 for support of these efforts  i  m at x 30352 if you have any questions  thanks 
分析：
1. 主旨與目的：該郵件的主旨是請求 Kevin 和 Tim 檢視附上的專案清單，並確認其是否同意所列出的項目與金額。
2. 語言特徵：郵件語言直接且清晰，使用了正式且專業的語氣，未包含過於吸引人的措辭或過多的呼籲性語句。
3. 潛在釣魚信特徵：
- 未包含任何不可信的 URL。
- 未要求接收者提供敏感信息（例如賬戶密碼或個人身份資料）。
- 未強調任何緊急性或試圖操控情緒。
4. 結論：「False」，此郵件是安全郵件，未包含任何釣魚信件特徵（False）。
```
"""
)

HumanPrompt = HumanMessagePromptTemplate.from_template(
"""
郵件：{input}
"""
)

CoT_prompt = ChatPromptTemplate.from_messages([
    SystemPrompt,
    HumanPrompt,
    # MessagesPlaceholder(variable_name="agent_scratchpad")
])

CoT_chain = (
    {"input": RunnablePassthrough()}
    | CoT_prompt
    | llm
)

In [ ]:
# 將結果存入 list
CoT_result = []
CoT_ans = []
# 使用for迴圈遍歷phishing_emails的"Email Text"欄位
# 將是否答對存進 list 中，而非回答的答案

for email_text in phishing_emails["Email Text"]:
    result = CoT_chain.invoke({"input": email_text})
    result = result.content
    CoT_result.append(result)
    if "true" in result.strip().lower():
        CoT_ans.append("True")
    else:
        CoT_ans.append("False")
    print(len(CoT_ans))

for email_text in safe_emails["Email Text"]:
    result = CoT_chain.invoke({"input": email_text})
    result = result.content
    CoT_result.append(result)
    if "false" in result.strip().lower():
        CoT_ans.append("True")
    else:
        CoT_ans.append("False")
    print(len(CoT_ans))

In [53]:
CoT_result

['1. 主旨與目的：該郵件旨在推銷一個與 WSPN 相關的投資或股票資訊，並引導收件人訪問一個網址以獲取更多資訊。\n\n2. 語言特徵：郵件內容包含大量數據和財務資訊，語言較為直接，試圖吸引投資者注意。使用了「預計擁有20萬訂閱者」、「預計收入1.2億美元」等吸引投資者的措辭，但沒有過度誇張或過多的呼籲性語句。\n\n3. 潛在釣魚信件特徵：\n- 提供了 URL： http finance yahoo com q s wspnd t 2314，這個網址格式不完整且不可信，可能是用來誘導點擊或收集個人資訊。\n- 試圖促使收件人訪問網址，可能存在釣魚或詐騙風險。\n- 沒有明確的公司聯絡資訊或合法證明，且網址格式不符合正式網站標準。\n\n4. 結論：「True」，此郵件具有典型的釣魚特徵，包括不可信的網址和誘導點擊的行為，因此判斷為釣魚信件（True）。',
 '1. 主旨與目的：該郵件的內容似乎是想吸引用戶點擊或回應，聲稱可以幫助支付嬰兒尿布費用，並暗示這對家庭有幫助，但沒有明確的詳細資訊或行動指示。\n\n2. 語言特徵：語言非常簡短且不正式，使用了過於吸引人的措辭（例如「幫你支付嬰兒尿布費用一年」），並且句子結構混亂，缺乏專業性，這是釣魚郵件常見的特徵。\n\n3. 潛在釣魚信件特徵：\n- 提供了極具吸引力的承諾（免費支付尿布費用一年），這是典型的誘餌策略。\n- 郵件內容非常模糊，沒有提供任何可信的聯絡方式或詳細資訊。\n- 內容中包含亂碼（"go odzzfzzq"），可能是故意用來迷惑或測試郵件過濾系統。\n- 沒有提供可信的網址或聯絡方式，也沒有明確的行動呼籲。\n\n4. 結論：「True」，此郵件具有典型的釣魚郵件特徵，包括過於誘人的承諾、內容模',
 '1. 主旨與目的：該郵件內容沒有明確的主旨或目的，似乎是隨機拼湊的文字，沒有具體的訊息或行動請求。\n2. 語言特徵：語言混亂且缺乏連貫性，包含一些看似隨機的詞彙和短語（例如「please solve your impotence」、「harvard medical」、「learn more」），但整體缺乏邏輯或明確的推銷內容。\n3. 潛在釣魚信特徵：\n- 內容中出現了誘導點擊的詞語（如「learn more」），但沒有提供有效的連結或明確的行動指示。\n- 內容極不專業且混亂，沒有提

In [61]:
CoT_acc = calcu_result(CoT_ans)
print(CoT_acc)

0.74


### RAG

#### embedding 一次就好

In [65]:
# 載入 PDF
loader = PyPDFLoader("./dataset/PhishingMail_Analysis.pdf")
pages = loader.load()

# 切成 chunk
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=200)
split_docs = text_splitter.split_documents(pages)


# 建立向量資料庫
# 將 embedding 資料存進向量資料庫
rag_database = "./chroma_db_small"
embedding_model="text-embedding-3-small"
embedding = OpenAIEmbeddings(model=embedding_model)

vectorstore = Chroma.from_documents(
    split_docs,
    embedding,
    persist_directory=rag_database
)
vectorstore.persist()

C:\Users\win90\AppData\Local\Temp\ipykernel_23420\1751797664.py:14: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(model=embedding_model)
C:\Users\win90\AppData\Local\Temp\ipykernel_23420\1751797664.py:21: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


#### 載入向量資料庫

In [83]:
# embedding_model="text-embedding-3-large"
# rag_database = "./chroma_db_large"
embedding_model="text-embedding-3-small"
rag_database = "./chroma_db_small"

embedding = OpenAIEmbeddings(model=embedding_model)
# 載入資料庫
vectorstore = Chroma(
    persist_directory=rag_database,  # 同樣的資料夾
    embedding_function=embedding
)

# k=3 代表取出 3 個最相關的 chunk
pdf_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})  # k=3 代表取出 3 個最相關的 chunk

rag_phishing = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=pdf_retriever,
    return_source_documents=True
)

# 根據 description 決定要使用的工具
rag_phishing_tool = Tool(
    name="pdfRAGRetriever", 
    func=rag_phishing.invoke, 
    description="內容包含釣魚信件的分析，請根據郵件內容進行檢索"
)

tools = [rag_phishing_tool]

In [84]:
pdf_retriever.invoke("free")

[Document(metadata={'author': 'Vivian Chien', 'creationdate': '2024-06-04T07:26:43+00:00', 'creator': 'Canva', 'keywords': 'DAGGq2xv-Cc,BAFtSmNEU9g', 'moddate': '2024-06-04T07:26:37+00:00', 'page': 16, 'page_label': '17', 'producer': 'Canva', 'source': './dataset/PhishingMail_Analysis.pdf', 'title': '悲傷Peter與他的快樂小伙伴-期末專案簡報', 'total_pages': 27}, page_content='釣\n⿂\n信\n件情緒代表字\nLexicon由圖表結果可看出：\n釣⿂信件的正⾯詞彙free, benefit, bonus⽐安全信件的please, well更加吸引⼈， 釣⿂信件可能是利⽤⼈類的貪念誘騙上當。\n⽽在負⾯詞彙中，釣⿂信件⽤了相較於安全信件負⾯詞彙第⼀名的 problem情緒更強烈的risk, stress，讓⼈更擔⼼受怕。\n釣\n⿂\n信\n件 (no.14275)\n安\n全\n信\n件\n︵ no.12501)\n安\n全\n信\n件'),
 Document(metadata={'author': 'Vivian Chien', 'creationdate': '2024-06-04T07:26:43+00:00', 'creator': 'Canva', 'keywords': 'DAGGq2xv-Cc,BAFtSmNEU9g', 'moddate': '2024-06-04T07:26:37+00:00', 'page': 4, 'page_label': '5', 'producer': 'Canva', 'source': './dataset/PhishingMail_Analysis.pdf', 'title': '悲傷Peter與他的快樂小伙伴-期末專案簡報', 'total_pages': 27}, page_content='詞頻分析\n分類觀察：總信件、安全信件、釣⿂信件計算去除停⽤字後的斷詞詞頻\nenr

In [106]:
SystemPrompt = SystemMessagePromptTemplate.from_template(
"""
你是一位資安專家，具有豐富的釣魚信件分析經驗。  
當接收到一封郵件內容時，請根據下列工具（{tools}）所檢索到的報告資料，逐步分析郵件是否為釣魚信件。  
分析流程： 
1. 用詞比對：比對郵件內容與釣魚信件高頻詞
2. 情緒評估：判斷郵件的整體情緒與釣魚常用情緒手法是否吻合
3. 主題驗證：核對郵件主題與已知釣魚主題模型結果
4. 根據分析給出結論，請回答「True」或「False」即可""")

HumanPrompt = HumanMessagePromptTemplate.from_template(
"""
郵件：{input}
"""
)

tools_str = "\n".join([f"- {tool.name}: {tool.description}" for tool in tools])

rag_prompt = ChatPromptTemplate.from_messages([
    SystemPrompt,
    HumanPrompt,
    MessagesPlaceholder(variable_name="agent_scratchpad")
]).partial(tools=tools_str)


# 創建 agent
rag_chain = create_openai_functions_agent(llm, tools, rag_prompt)

# 創建 AgentExecutor
agent_executor = AgentExecutor(
    agent=rag_chain,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [107]:
# 將結果存入 list
rag_result = []
rag_ans = []
# 使用for迴圈遍歷phishing_emails的"Email Text"欄位
# 將是否答對存進 list 中，而非回答的答案

for email_text in phishing_emails["Email Text"]:
    result = agent_executor.invoke({"input": email_text})
    result = result['output']
    rag_result.append(result)
    if "true" in result.strip().lower():
        rag_ans.append("True")
    else:
        rag_ans.append("False")
    print(len(rag_ans))

for email_text in safe_emails["Email Text"]:
    result = agent_executor.invoke({"input": email_text})
    result = result['output']
    rag_result.append(result)
    if "false" in result.strip().lower():
        rag_ans.append("True")
    else:
        rag_ans.append("False")
    print(len(rag_ans))



> Entering new AgentExecutor chain...

Invoking: `pdfRAGRetriever` with `郵件內容包含大量促銷和投資相關詞彙，並且誘導點擊連結，具有典型的釣魚信件特徵。`


{'query': '郵件內容包含大量促銷和投資相關詞彙，並且誘導點擊連結，具有典型的釣魚信件特徵。', 'result': '根據提供的資料分析，郵件內容包含大量促銷和投資相關詞彙，並誘導點擊連結，確實符合釣魚信件的典型特徵。釣魚信件常利用優惠、金錢誘因，以及網址連結來吸引收信者點擊，並且多數屬於金融投資或免費軟體等類型。建議提高警覺，避免點擊可疑連結，以保障個人資訊安全。', 'source_documents': [Document(metadata={'author': 'Vivian Chien', 'creationdate': '2024-06-04T07:26:43+00:00', 'creator': 'Canva', 'keywords': 'DAGGq2xv-Cc,BAFtSmNEU9g', 'moddate': '2024-06-04T07:26:37+00:00', 'page': 12, 'page_label': '13', 'producer': 'Canva', 'source': './dataset/PhishingMail_Analysis.pdf', 'title': '悲傷Peter與他的快樂小伙伴-期末專案簡報', 'total_pages': 27}, page_content='主題模型結論 \nLDA、 GuideLDA 、 BERTopic\n透過主題分析，我們得出以下推論：\n釣⿂信常常出現如  company 、 free 、 get 、 click 、 money 等，⽬的是透過\n優惠和⾦錢來引誘收信者點擊連結。\n釣⿂信通常包含網址連結或網⾴程式碼。\n釣⿂信⼤致分為免費軟體、⾦融投資、醫療藥品及給予⾦錢這四種常⾒類型。\n通過主題分析，我們得以更深⼊了解釣⿂信的特徵，從⽽更有效地辨識釣⿂信件。'), Document(metadata={'author': 'Vivian Chien', 'creationdate': '2024-06-04T07:26:43+00:0

In [100]:
rag_result

['False\n\n根據分析，該郵件具有潛在的釣魚特徵，包括誘導點擊的連結、商業化語言以及缺乏正式公司資料，符合釣魚信的典型特徵。然而，該郵件並未明確要求提供敏感信息，也沒有直接威脅或緊急性提示，因此不能完全確定為釣魚郵件，但仍具有高度可疑性。',
 '這封郵件的內容具有釣魚信的特徵，特別是提到「get your babies diapers bill paid for for a year」這類與金錢和優惠相關的誘因，並且內容中包含促使收件人點擊連結的誘導語。此外，詞頻分析結果也顯示此類郵件常包含「free」、「get」、「click」、「money」等詞彙，這些都是釣魚信的典型特徵。因此，這封郵件很可能是釣魚信件，具有誘導點擊和騙取敏感資訊的風險。\n\n結論：True',
 'False\n\n理由：該郵件內容包含了大量無意義的詞彙和短語，並且使用了誘導性語言（如“please solve your impotence”），試圖引起收件人的注意或焦慮。此外，內容中提到的“harvard medical”可能是模仿可信機構的名稱，並且沒有提供任何合法或可信的連結或指示。根據分析，這封郵件具有典型的釣魚信件特徵，包括不合理的內容、誘導性語言和潛在的惡意意圖。因此，這是一封釣魚郵件。',
 'True\n\n根據檢索結果，該郵件具有多個釣魚信件的典型特徵，包括：\n- 使用誘人的金融優惠（如低利率貸款、無需信用審查、快速批准）來吸引收件人點擊連結。\n- 強調「快速」、「簡單」、「少於2分鐘」等緊迫感和便利性，誘使收件人忽略警覺。\n- 提供不可信的URL（http://www.dfgsgs.info/443/azecwbzmi），這類可疑域名常用於釣魚。\n- 整體語言偏向過度促銷，缺乏正式的專業語氣。\n\n資料分析也確認此類郵件屬於釣魚信件範疇，具有明顯的誘騙和欺詐特徵。因此，這封郵件很可能是釣魚郵件。',
 'False\n\n根據分析，這封郵件內容沒有展現出典型的釣魚信件特徵，例如請求敏感信息、提供不可信的URL、強調緊急性或使用過於吸引人的措辭。內容較為隨意，像是拼湊的引述或隨筆，缺乏誘騙元素，因此判斷這封郵件的釣魚風險較低。',
 '根據分析結果，這封郵件的內容主要涉及醫療藥品和網頁程式碼相關的主題，並沒有明顯的釣魚信件特徵，例如要求提供敏感資訊、提

In [108]:
rag_acc = calcu_result(rag_ans)
print(rag_acc)

0.76


### Result

In [110]:
# 建立數據
data = {
    "Method": ["Normal", "Few-shot", "CoT", "RAG"],
    "Accuracy": [normal_acc, fewshot_acc, CoT_acc, rag_acc]
}

# 轉換為 DataFrame
df = pd.DataFrame(data)

# 顯示表格
print(df)

     Method  Accuracy
0    Normal      0.72
1  Few-shot      0.66
2       CoT      0.74
3       RAG      0.76
